# Partie Comptage de produit



Dans ce notebook, on a la partie de comptage des produits detectés et classifié par le modèle precédent, leur séparation est juste une question de structuration pour faciliter la comprehension, ce modele fera appel au modele precédent qui est importé (il est deposé sur le drive)

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 38.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
import os
import torch
import cv2
import numpy as np
from PIL import Image
from collections import defaultdict
from torchvision import transforms, models
from ultralytics import YOLO

# 📍 CONFIGURATION
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 🏷️ Chargement du modèle YOLOv8
yolo_model = YOLO("yolov8s.pt")

# 🏷️ Chargement du modèle de classification
MODEL_PATH = "resnet18_classification.pth"
classification_model = models.resnet18(weights=True)

# le nombre de classes réelles
num_classes = 5
classification_model.fc = torch.nn.Linear(classification_model.fc.in_features, num_classes)
classification_model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
classification_model.to(device)
classification_model.eval()

# 🔄 Prétraitement des images pour la classification
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 🏷️ Liste des classes (doit correspondre aux classes du modèle de classification)
class_names = ["autre produit", "bouteille_ramy", "bouteille_rouiba", "cannette_ramy", "pack_rouiba"]  # ⚠️ À personnaliser

# 🚨 Classes que nous voulons détecter avec YOLO (bouteilles et canettes uniquement)
allowed_classes = [39, 41]  # Bottle (39), Can (41)

# 📌 Fonction principale
def count_products(image_path, detected_products_dir="/content/detected_products"):
    """
    📌 Détecte et compte les bouteilles et canettes dans une image.
    ➡️ Renvoie un dictionnaire avec le nombre d'occurrences par classe.

    :param image_path: Chemin de l'image d'entrée
    :param detected_products_dir: Dossier où sauvegarder les crops des produits détectés
    :return: Dictionnaire {nom_classe: nombre_detecté}
    """
    # Assurer que le dossier de stockage des crops existe
    os.makedirs(detected_products_dir, exist_ok=True)

    # 📌 1. Détection des objets avec YOLO
    image = cv2.imread(image_path)

    results = yolo_model(image_path)
    # Sauvegarder les résultats
    cv2.imwrite("resultat.jpg", results[0].plot())
    # 📌 2. Initialiser un dictionnaire pour compter les produits
    product_counts = defaultdict(int)

    # 📌 3. Extraction et classification des produits détectés
    for i, box in enumerate(results[0].boxes.xyxy):
        class_id = int(results[0].boxes.cls[i])  # ID de la classe détectée

        # 🚨 Filtrer les objets non souhaités (seulement bouteilles et canettes)
        if class_id not in allowed_classes:
            print(f"❌ Produit {i} ignoré (Classe {class_id} non prise en charge)")
            continue

        # 📌 Si l'objet est valide, extraire et sauvegarder
        x1, y1, x2, y2 = map(int, box)
        cropped = image[y1:y2, x1:x2]

        # 📌 Sauvegarde du crop
        crop_path = os.path.join(detected_products_dir, f"product_{i}.jpg")
        cv2.imwrite(crop_path, cropped)

        # 📌 4. Classification du produit détecté
        image_pil = Image.open(crop_path).convert('RGB')
        image_tensor = transform(image_pil).unsqueeze(0).to(device)

        with torch.no_grad():
            output = classification_model(image_tensor)
            _, predicted = output.max(1)

        predicted_class = class_names[predicted.item()]
        product_counts[predicted_class] += 1  # Incrémenter le compteur

        print(f"✅ Produit {i} détecté (Classe COCO {class_id}) et classé comme {predicted_class}")

    # 📊 5. Résumé du comptage des produits
    print("\n📊 RÉSUMÉ DU COMPTAGE :")
    for class_name, count in product_counts.items():
        print(f"🛒 {class_name} : {count} produits")

    return dict(product_counts)

# 🛠️ **Utilisation**
image_path = "/content/5.jpg"
results = count_products(image_path)

# 🔍 Afficher le dictionnaire final
print("\n📌 Résultat final :", results)



image 1/1 /content/5.jpg: 480x640 47 bottles, 1 refrigerator, 483.8ms
Speed: 5.1ms preprocess, 483.8ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)
✅ Produit 0 détecté (Classe COCO 39) et classé comme autre produit
✅ Produit 1 détecté (Classe COCO 39) et classé comme autre produit
✅ Produit 2 détecté (Classe COCO 39) et classé comme autre produit
✅ Produit 3 détecté (Classe COCO 39) et classé comme autre produit
✅ Produit 4 détecté (Classe COCO 39) et classé comme autre produit
✅ Produit 5 détecté (Classe COCO 39) et classé comme autre produit
✅ Produit 6 détecté (Classe COCO 39) et classé comme autre produit
✅ Produit 7 détecté (Classe COCO 39) et classé comme autre produit
✅ Produit 8 détecté (Classe COCO 39) et classé comme bouteille_ramy
✅ Produit 9 détecté (Classe COCO 39) et classé comme autre produit
✅ Produit 10 détecté (Classe COCO 39) et classé comme autre produit
✅ Produit 11 détecté (Classe COCO 39) et classé comme autre produit
✅ Produit 12 détecté (Cl